# Environment

### Imports

In [ ]:
import gym
from gym import Env, spaces

### Definición

In [ ]:
class Estudiante(object):
    def __init__(self, id, preferencias = []):
        self.id = id
        self.preferencias = preferencias # IDs de desafíos

class Desafio(object):
    def __init__(self, id, estudiantes : list = []):
        self.id = id
        self.estudiantes = estudiantes # IDs de estudiantes

    def push(self, estudiante):
        self.estudiantes.append(estudiante)

In [ ]:
num_desafios = 5
num_estudiantes = 10


class OrganizationEnvironment(Env):
    def __init__(self):
        super(OrganizationEnvironment, self).__init__()
        
        # Definición de espacio de observación
        self.observation_space = spaces.Dict({
            "desafios" : spaces.Tuple((
                spaces.Discrete(num_desafios), # Número máximo de desafíos (ID desafío)
                spaces.MultiBinary(num_estudiantes) # Número máximo de estudiantes (IDs de estudiantes que participan en los desafíos)
            )),
            "estudiantes" : spaces.Tuple((
                spaces.Discrete(num_estudiantes), # Número máximo de estudiantes (ID estudiante)
                spaces.MultiDiscrete([num_desafios,num_desafios,num_desafios]) # Preferencias ordenadas por
            ))
        })
        
        # Definición de espacio de acción
        self.action_space = spaces.MultiDiscrete([num_estudiantes, num_desafios])
    
    def reset(self):
        pass
    
    def step(self, action):
        pass